## A. Installation

### A.1. Structure

Pour réinitialiser la structure (from scratch) :

In [1]:
!cd /content
!rm -rf /content/Merval
!git clone https://github.com/mervealgan/Merval

Cloning into 'Merval'...
remote: Enumerating objects: 35, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 35 (delta 12), reused 29 (delta 11), pack-reused 0 (from 0)
Receiving objects: 100% (35/35), 366.01 KiB | 1.58 MiB/s, done.
Resolving deltas: 100% (12/12), done.


In [2]:
!rm -rf /content/training/data
!mkdir -p /content/training/data/features

!cp -r /content/Merval/data/features/* /content/training/data/features
!cp /content/Merval/data/test_set.csv /content/training/data/test_set.csv
!cp /content/Merval/data/training_set.csv /content/training/data/training_set.csv
!cp /content/Merval/data/valid_set.csv /content/training/data/valid_set.csv

In [3]:
%cd /content/training/

/content/training


### A.2. imports

In [4]:
!pip install tbparse
!pip install syntok
!pip install stanza
!pip install textcomplexity
!pip install transformers[torch]
!pip install accelerate -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.1/990.1 kB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 20.6 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.32.1
    Uninstalling accelerate-0.32.1:
      Successfully uninstalled accelerate-0.32.1


## B. Entrainement

In [5]:
import hashlib
import os
import torch
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import (Sequential, load_model)
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from transformers import (AutoTokenizer, TrainingArguments, AutoModelForSequenceClassification, set_seed, Trainer,
                          EarlyStoppingCallback, )


class TCCDataset(torch.utils.data.Dataset):
    def __init__(self, tokens, labels):
        self.tokens = tokens
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        # return tensor
        item = {key: val[idx].clone().detach() for key, val in self.tokens.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item


class RegressionTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fct = torch.nn.MSELoss()
        loss = loss_fct(
            logits.view(-1, self.model.config.num_labels),
            labels.float().view(-1, self.model.config.num_labels),
        )
        return (loss, outputs) if return_outputs else loss


class OptimizedESCallback(EarlyStoppingCallback):
    def __init__(self, patience, initial_steps_wo_save):
        super().__init__(early_stopping_patience=patience)
        self.initial_steps_wo_save = initial_steps_wo_save

    def check_metric_value(self, args, state, control, metric_value):
        super().check_metric_value(args, state, control, metric_value)
        if self.early_stopping_patience_counter == 0:
            control.should_save = True

    def on_evaluate(self, args, state, control, metrics, **kwargs):
        if state.global_step < self.initial_steps_wo_save:
            return
        super().on_evaluate(args, state, control, metrics, **kwargs)


def compute_metrics(y_true, y_pred):
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)

    return {
        "root_mean_squared_error": rmse,
        "mean_absolute_error": mae,
        "mean_squared_error": mse,
    }


def compute_metrics_for_regression(eval_pred):
    logits, labels = eval_pred
    labels = labels.reshape(-1, 1)

    rmse = mean_squared_error(labels, logits, squared=False)
    mse = mean_squared_error(labels, logits)
    mae = mean_absolute_error(labels, logits)

    return {
        "root_mean_squared_error": rmse,
        "mean_absolute_error": mae,
        "mean_squared_error": mse,
    }


def get_hugging_face_name(name):
    if name == "camembert-base":
        return "almanach/camembert-base"
    if name == "camembert-large":
        return "almanach/camembert-large"
    return ""


def load_dataset(path, encoding="utf-8", shuffle=True):
    df = pd.read_csv(path, encoding=encoding)
    df.drop_duplicates(inplace=True)
    if shuffle:
        df = df.sample(frac=1, random_state=9).reset_index(drop=True)
    return df


def load_dataset_with_features_fr(dataset, data_root_path='data_fr'):
    df = load_dataset(os.path.join(data_root_path, f'{dataset}_set.csv'))
    df_features = pd.read_csv(os.path.join(data_root_path, 'features', f'features_{dataset}_readability_fr.csv'))
    df_merged = df.merge(df_features, on='ID', suffixes=('', '_df2'))

    # drop or ignore some columns
    ignore_columns = ['sentence_df2', 'paragraphs', 'sentences_per_paragraph']
    df_merged.drop(columns=ignore_columns, inplace=True)

    # add some of our own features
    df_merged['max_word_length'] = df_merged['sentence'].apply(lambda x: max([len(w) for w in x.split()]))

    for i in range(5, 10):
        df_merged['num_word_longer_than_' + str(i)] = df_merged['sentence'].apply(
            lambda x: sum([len(w) > i for w in x.split()]))

    feature_columns = df_merged.columns.to_list()[df_merged.columns.to_list().index('sentence') + 1:]

    return df_merged, feature_columns


os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

for gpu in tf.config.experimental.list_physical_devices('GPU'):
    tf.config.experimental.set_memory_growth(gpu, True)

BOOTSTRAP_SIZE = 1000  # 1000
MAX_ENSEMBLE_SIZE = 35  # 60
ENSEMBLE_POOL_SIZE = 40  # 100
N_FOLDS = 5
MODEL_NAME = 'camembert-base'  # ['gbert', 'gelectra', 'gottbert', 'gerpt']
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 16
N_EVAL_STEPS = 23

EXPERIMENT_NAME = f'ensemble_{MODEL_NAME}'
EXPERIMENT_DIR = f'cache/{EXPERIMENT_NAME}'

from tensorflow.keras.callbacks import TensorBoard
log_dir = f'{EXPERIMENT_DIR}/logs/mlp/'
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

df_train, feature_columns = load_dataset_with_features_fr('training', data_root_path='data')


def get_predictions(
        df_train_folds,
        df_val_fold,
        n_epochs=10, # Increased from 5 to 10
        n_log_steps=10,
):
    tf.debugging.disable_traceback_filtering()
    # storing predictions in dataframe
    # columns: Sentence, Prediction of Model 1, Prediction of Model 2, ...
    df_predictions_val_fold = df_val_fold[['ID', 'sentence']].copy()

    # get tokenizer
    tokenizer = AutoTokenizer.from_pretrained(get_hugging_face_name(MODEL_NAME))

    X_val_fold = df_val_fold['sentence'].values
    X_val_fold_features = df_val_fold[feature_columns].values

    # tokenize
    tokens_val_fold = tokenizer(X_val_fold.tolist(), padding='max_length', return_tensors='pt', truncation=True,
                                max_length=128)

    for k in range(ENSEMBLE_POOL_SIZE):
        df_early_stopping = df_train_folds.sample(frac=0.1, random_state=k)
        df_train_no_es = df_train_folds.drop(
            df_train_folds[
                df_train_folds['ID'].isin(df_early_stopping['ID'])
            ].index
        )

        ## or use this simplified code to drop rows whose 'ID' is in df_early_stopping['ID']
        # df_train_no_es = df_train_folds[~df_train_folds['ID'].isin(df_early_stopping['ID'])]

        X_early_stopping = df_early_stopping['sentence'].values
        X_early_stopping_features = df_early_stopping[feature_columns].values
        y_early_stopping = df_early_stopping['MOS'].values

        X_training = df_train_no_es['sentence'].values
        X_training_features = df_train_no_es[feature_columns].values
        y_training = df_train_no_es['MOS'].values

        # tokenize
        tokens_early_stopping = tokenizer(X_early_stopping.tolist(), padding='max_length', return_tensors='pt',
                                          truncation=True, max_length=128)

        tokens_training = tokenizer(X_training.tolist(), padding='max_length', return_tensors='pt', truncation=True,
                                    max_length=128)

        hash = (
                hashlib.sha256(
                    pd.util.hash_pandas_object(df_train_no_es['ID'], index=True).values
                ).hexdigest()
                + '_'
                + get_hugging_face_name(MODEL_NAME)[
                  get_hugging_face_name(MODEL_NAME).find('/') + 1:
                  ]
        )

        # load model and, if necessary, train it
        try:
            print(f'{EXPERIMENT_DIR}/models/{MODEL_NAME}/{hash}')
            model = AutoModelForSequenceClassification.from_pretrained(
                f'{EXPERIMENT_DIR}/models/{MODEL_NAME}/{hash}', local_files_only=True, num_labels=1
            )
        except EnvironmentError:
            # create training dataset
            early_stopping_dataset = TCCDataset(tokens_early_stopping, y_early_stopping)
            training_dataset = TCCDataset(tokens_training, y_training)

            training_args = TrainingArguments(
                output_dir=f'{EXPERIMENT_DIR}/{MODEL_NAME}_trainer/',
                num_train_epochs=10, # Updated (n_epochs)
                per_device_train_batch_size=TRAIN_BATCH_SIZE,
                per_device_eval_batch_size=VALID_BATCH_SIZE,
                warmup_ratio=0.3,
                learning_rate=5e-5,
                no_cuda=False,
                metric_for_best_model='root_mean_squared_error',
                greater_is_better=False,
                load_best_model_at_end=True,
                save_steps=N_EVAL_STEPS * 100_000,
                # we never want to save a model through this function, but the parameter must be set, because of load_best_model_at_end=True
                save_total_limit=1,  # can be 1, because we only save, when we find a better model
                eval_steps=N_EVAL_STEPS,
                # `evaluation_strategy` is deprecated, Use `eval_strategy` instead
                eval_strategy='steps',
                seed=k,
                logging_steps=n_log_steps,
                logging_dir=f'{EXPERIMENT_DIR}/logs/member_{k}',
                logging_strategy='steps',
            )

            set_seed(training_args.seed)
            model = AutoModelForSequenceClassification.from_pretrained(
                get_hugging_face_name(MODEL_NAME), num_labels=1
            )

            trainer = RegressionTrainer(
                model=model,
                args=training_args,
                train_dataset=training_dataset,
                eval_dataset=early_stopping_dataset,
                compute_metrics=compute_metrics_for_regression,
                callbacks=[OptimizedESCallback(patience=5, initial_steps_wo_save=300)],
            )
            # training
            trainer.train()

            # save model
            model.save_pretrained(f'{EXPERIMENT_DIR}/models/{MODEL_NAME}/{hash}')

        # load hidden states of model for validation and test data
        hidden_state_val_fold = extract_hidden_state(model, tokens_val_fold)

        # normalize data with StandardScaler
        scaler = StandardScaler()
        scaler.fit(df_train_folds[feature_columns].values)
        X_val_fold_features_scaled = scaler.transform(X_val_fold_features)
        X_val_fold_with_features = np.concatenate((hidden_state_val_fold.detach().numpy(), X_val_fold_features_scaled),
                                                  axis=1)

        # load MLP model and, if necessary, train it
        try:
            mlp = load_model(f'{EXPERIMENT_DIR}/models/mlp/{hash}_mlp.keras')
        except Exception:
            hidden_state_train = extract_hidden_state(model, tokens_training)
            hidden_state_early_stopping = extract_hidden_state(model, tokens_early_stopping)

            np.random.seed(k)
            mlp = Sequential(
                [
                    Input(shape=(model.config.hidden_size + len(feature_columns),), name='input'),
                    Dense(model.config.hidden_size, activation='relu', name='layer1'),
                    Dense(1, activation='linear', name='layer2'),
                ]
            )

            mlp.compile(
                optimizer='rmsprop',
                loss=tf.keras.losses.MeanSquaredError(),
                metrics=[tf.keras.metrics.RootMeanSquaredError()],
            )
            es = EarlyStopping(monitor='val_root_mean_squared_error', mode='min', verbose=1, patience=100)
            mc = ModelCheckpoint(
                f'{EXPERIMENT_DIR}/models/mlp/{hash}_mlp.keras',
                 monitor='val_root_mean_squared_error',
                 mode='min',
                 verbose=1,
                 save_best_only=True
            )

            # normalize data with StandardScaler
            scaler = StandardScaler()
            scaler.fit(X_training_features)
            X_train_features_scaled = scaler.transform(X_training_features)
            X_es_features_scaled = scaler.transform(X_early_stopping_features)

            X_train_with_features = np.concatenate((hidden_state_train.detach().numpy(), X_train_features_scaled),
                                                   axis=1)
            X_es_with_features = np.concatenate((hidden_state_early_stopping.detach().numpy(), X_es_features_scaled),
                                                axis=1)

            mlp.fit(X_train_with_features, y_training,
                    validation_data=(X_es_with_features, y_early_stopping),
                    batch_size=TRAIN_BATCH_SIZE,
                    #epochs=5000, callbacks=[es, mc])
                    epochs=10, callbacks=[tensorboard_callback, es, mc])

        # Manually save the model after training
        mlp.save(f'{EXPERIMENT_DIR}/models/mlp/{hash}_mlp.keras')

        mlp = tf.keras.models.load_model(f'{EXPERIMENT_DIR}/models/mlp/{hash}_mlp.keras')

        # predict MLP on validation and test sets
        prediction_val_fold = mlp.predict(X_val_fold_with_features, batch_size=VALID_BATCH_SIZE)

        df_predictions_val_fold[f'{MODEL_NAME}_prediction_{k}'] = prediction_val_fold

    return df_predictions_val_fold


def extract_hidden_state(model, tokens, batch_size=16):
    last_last_hidden_state = torch.zeros((len(tokens.input_ids), model.config.hidden_size))
    model = model.cuda().eval()
    with torch.no_grad():
        for i in range(0, len(tokens.input_ids), batch_size):
            if i + batch_size > len(tokens.input_ids):
                input_i = tokens.input_ids[i:]
            else:
                input_i = tokens.input_ids[i:i + batch_size]
            output = model(input_i.cuda(), output_hidden_states=True)
            last_hidden_state = output.hidden_states[-1].cpu()
            idx_last_token = torch.zeros(len(input_i)).long()
            last_last_hidden_state[i:i + len(idx_last_token)] = last_hidden_state[
                torch.arange(len(idx_last_token)), idx_last_token]
    return last_last_hidden_state


# dataframe for each metric for each model for each ensemble size
# 3d array: [ensemble_size, model_index, metric_index]
df_macro_ensemble_scores = pd.DataFrame(
    columns=[
        'ensemble_size',
        'model_name',
        'mean_absolute_error_mean',
        'mean_absolute_error_std',
        'mean_squared_error_mean',
        'mean_squared_error_std',
        'root_mean_squared_error_mean',
        'root_mean_squared_error_std',
    ]
)

for fold, (train_idx, val_idx) in enumerate(KFold(n_splits=N_FOLDS).split(df_train)):
    df_train_folds = df_train.loc[train_idx]
    df_val_fold = df_train.loc[val_idx]
    # fill na with mean of columns of train data
    df_train_folds = df_train_folds.fillna(df_train_folds.mean(numeric_only=True))
    df_val_fold = df_val_fold.fillna(df_train_folds.mean(numeric_only=True))

    y_val_fold = df_val_fold['MOS'].values

    pool_predictions_val_fold = get_predictions(df_train_folds, df_val_fold)

    for current_ensemble_size in range(1, MAX_ENSEMBLE_SIZE + 1):
        np.random.seed(current_ensemble_size)
        idx = np.random.choice(
            ENSEMBLE_POOL_SIZE,
            size=(BOOTSTRAP_SIZE, current_ensemble_size),
        )

        idx_mapped = np.array(
            [
                np.array(
                    [pool_predictions_val_fold[f'{MODEL_NAME}_prediction_{k}'] for k in j]
                )
                for j in idx
            ]
        )

        ensemble_predictions = np.array(
            [np.sum(j, axis=0) / len(j) for j in idx_mapped]
        )

        ensemble_scores = [
            compute_metrics(y_val_fold, pred) for pred in ensemble_predictions
        ]

        df_ensemble_scores = pd.DataFrame(ensemble_scores).sort_index(axis=1)

        # add to dataframe
        new_row = pd.DataFrame(
            {
                'ensemble_size': [current_ensemble_size],
                'model_name': [MODEL_NAME],
                'mean_absolute_error_mean': [df_ensemble_scores['mean_absolute_error'].mean()],
                'mean_absolute_error_std': [df_ensemble_scores['mean_absolute_error'].std()],
                'mean_squared_error_mean': [df_ensemble_scores['mean_squared_error'].mean()],
                'mean_squared_error_std': [df_ensemble_scores['mean_squared_error'].std()],
                'root_mean_squared_error_mean': [df_ensemble_scores['root_mean_squared_error'].mean()],
                'root_mean_squared_error_std': [df_ensemble_scores['root_mean_squared_error'].std()],
            })

df_macro_ensemble_scores = pd.concat([df_macro_ensemble_scores, new_row], ignore_index=True)

df_macro_ensemble_scores[
    df_macro_ensemble_scores['model_name'] == MODEL_NAME
    ].to_csv(
    f'ensemble_scores_{MODEL_NAME}.csv', index=False, sep=';', encoding='utf-8'
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/508 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/811k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

...

Epoch 1/10
 1/18 ━━━━━━━━━━━━━━━━━━━━ 11s 681ms/step - loss: 10.2410 - root_mean_squared_error: 3.2002
Epoch 1: val_root_mean_squared_error improved from inf to 0.44718, saving model to cache/ensemble_camembert-base/models/mlp/2915e102be27d8ee21602ad49f1da4d8af2bca91b52c9aa804e29667dee2d601_camembert-base_mlp.keras
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 3.6381 - root_mean_squared_error: 1.8094 - val_loss: 0.2000 - val_root_mean_squared_error: 0.4472
Epoch 2/10
 1/18 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.3503 - root_mean_squared_error: 0.5919
Epoch 2: val_root_mean_squared_error improved from 0.44718 to 0.41485, saving model to cache/ensemble_camembert-base/models/mlp/2915e102be27d8ee21602ad49f1da4d8af2bca91b52c9aa804e29667dee2d601_camembert-base_mlp.keras
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.5099 - root_mean_squared_error: 0.7134 - val_loss: 0.1721 - val_root_mean_squared_error: 0.4149
Epoch 3/10
 1/18 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.1063 - root_m

Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at almanach/camembert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss,Root Mean Squared Error,Mean Absolute Error,Mean Squared Error
23,5.555600,3.126340,1.768146,1.592193,3.126340
46,2.372800,0.719175,0.848042,0.655244,0.719175
69,0.868700,0.580971,0.762214,0.602671,0.580971
92,0.828500,0.627451,0.792118,0.625249,0.627451
115,0.461700,0.680983,0.825217,0.610768,0.680983
138,0.475200,0.573852,0.757531,0.604477,0.573852
161,0.271400,0.601993,0.775882,0.624644,0.601993


Epoch 1/10
 1/18 ━━━━━━━━━━━━━━━━━━━━ 11s 661ms/step - loss: 5.1542 - root_mean_squared_error: 2.2703
Epoch 1: val_root_mean_squared_error improved from inf to 0.84615, saving model to cache/ensemble_camembert-base/models/mlp/a26faa62580a71129dba42331db1036b03fb5a20d296815b9af286c97f35a242_camembert-base_mlp.keras
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 2.2374 - root_mean_squared_error: 1.4325 - val_loss: 0.7160 - val_root_mean_squared_error: 0.8461
Epoch 2/10
 1/18 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.7233 - root_mean_squared_error: 0.8505
Epoch 2: val_root_mean_squared_error improved from 0.84615 to 0.65219, saving model to cache/ensemble_camembert-base/models/mlp/a26faa62580a71129dba42331db1036b03fb5a20d296815b9af286c97f35a242_camembert-base_mlp.keras
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.6205 - root_mean_squared_error: 0.7770 - val_loss: 0.4254 - val_root_mean_squared_error: 0.6522
Epoch 3/10
 1/18 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.6061 - root_me

<ipython-input-5-c47b25c633a3>:413: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_macro_ensemble_scores = pd.concat([df_macro_ensemble_scores, new_row], ignore_index=True)


In [ ]:
%rm -rf cache

In [6]:
from google.colab import files
!zip -r /content/logs.zip /content/training/cache/ensemble_camembert-base/logs

files.download('/content/logs.zip')

  adding: content/training/cache/ensemble_camembert-base/logs/ (stored 0%)
  adding: content/training/cache/ensemble_camembert-base/logs/member_1/ (stored 0%)
  adding: content/training/cache/ensemble_camembert-base/logs/member_1/events.out.tfevents.1725259041.2d88bd8a8071.326.1 (deflated 64%)
  adding: content/training/cache/ensemble_camembert-base/logs/member_1/events.out.tfevents.1725260762.2d88bd8a8071.326.41 (deflated 64%)
  adding: content/training/cache/ensemble_camembert-base/logs/member_1/events.out.tfevents.1725264315.2d88bd8a8071.326.121 (deflated 64%)
  adding: content/training/cache/ensemble_camembert-base/logs/member_1/events.out.tfevents.1725262515.2d88bd8a8071.326.81 (deflated 64%)
  adding: content/training/cache/ensemble_camembert-base/logs/member_1/events.out.tfevents.1725266151.2d88bd8a8071.326.161 (deflated 64%)
  adding: content/training/cache/ensemble_camembert-base/logs/member_26/ (stored 0%)
  adding: content/training/cache/ensemble_camembert-base/logs/member_2

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!cp -r /content/training/cache/ensemble_camembert-base/models /content/drive/MyDrive/Models0109

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
